In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder,OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
from sklearn.preprocessing import FunctionTransformer,PowerTransformer
import re
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB

In [2]:
data = pd.read_csv("spam (1).csv",encoding="latin-1",usecols=["v1","v2"])

In [3]:
df = data.copy()

In [4]:
df

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will Ì_ b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [5]:
fv = df.iloc[:,1]
cv = df.iloc[:,0]

In [6]:
cv= cv.map({"spam":0,"ham":1})

In [7]:
x_train,x_test,y_train,y_test = train_test_split(fv,cv,test_size=0.2,random_state=1,stratify= cv)

In [8]:
def lowers(x):
    return x.str.lower()

In [9]:
def html(x):
    return x.apply(lambda x:re.sub("<.+?>"," ",x))

In [10]:
def url(x):
    return x.apply(lambda x:re.sub('http[s]?://.+? +' ," ", x))

In [11]:
def unwanted(x):
    return x.apply(lambda x:re.sub("[^a-z\s]", " ", x))

In [12]:
pre_processing_pi=Pipeline([("lower",FunctionTransformer(lowers)),
         ("html",FunctionTransformer(html)),
         ("url",FunctionTransformer(url)),
         ("und",FunctionTransformer(unwanted))])

In [13]:
pre_processing_pi.fit_transform(x_train)

3539    i ll get there at    unless you guys want me t...
2009    i can t keep going through this  it was never ...
5363    i think your mentor is   but not     percent s...
3665    i m turning off my phone  my moms telling ever...
3441    save money on wedding lingerie at www bridal p...
                              ...                        
5360    hey  iouri gave me your number  i m wylie  rya...
2663         free for  st week  no  nokia tone   ur mo...
3163    today is sorry day   if ever i was angry with ...
1525                     pls pls find out from aunt nike 
3914               ard     lor  i ok then message    lor 
Name: v2, Length: 4457, dtype: object

In [14]:
final_pi = Pipeline([("pre_process",pre_processing_pi),("vectorizer",CountVectorizer())])

In [15]:
x_train_fit = final_pi.fit_transform(x_train)

In [16]:
x_test_t = final_pi.transform(x_test)

In [17]:
import pickle

In [18]:
pickle.dump(final_pi,open(r"C:\Users\Welcome\Machine Learning\Daily Tasks\final_spam.pkl","wb"))

In [19]:
mb = MultinomialNB()
model = mb.fit(x_train_fit,y_train)

In [20]:
pickle.dump(model,open(r"C:\Users\Welcome\Machine Learning\Daily Tasks\spam.pkl","wb"))

In [21]:
modelf = pickle.load(open(r"C:\Users\Welcome\Machine Learning\Daily Tasks\spam.pkl","rb"))

In [22]:
finalf = pickle.load(open(r"C:\Users\Welcome\Machine Learning\Daily Tasks\final_spam.pkl","rb"))

In [23]:
modelf

MultinomialNB()

In [24]:
finalf

Pipeline(steps=[('pre_process',
                 Pipeline(steps=[('lower',
                                  FunctionTransformer(func=<function lowers at 0x00000212A96676D0>)),
                                 ('html',
                                  FunctionTransformer(func=<function html at 0x00000212A9667AC0>)),
                                 ('url',
                                  FunctionTransformer(func=<function url at 0x00000212A9667E20>)),
                                 ('und',
                                  FunctionTransformer(func=<function unwanted at 0x00000212A96F01F0>))])),
                ('vectorizer', CountVectorizer())])

In [25]:
accuracy_score(y_test,model.predict(x_test_t))

0.9829596412556054

In [26]:
print(classification_report(y_test,model.predict(final_pi.transform(x_test))) )

              precision    recall  f1-score   support

           0       0.95      0.92      0.94       149
           1       0.99      0.99      0.99       966

    accuracy                           0.98      1115
   macro avg       0.97      0.96      0.96      1115
weighted avg       0.98      0.98      0.98      1115



In [27]:
m = final_pi.transform(pd.Series("i won the lottary")) 
pred = model.predict(m)
if pred == 0:
    print("spam")
else:
    print("ham")


ham
